In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import jaccard_distance

import warnings
warnings.filterwarnings('ignore')	


In [2]:
df_1 = pd.read_excel('../Data/products.xlsx')

In [3]:
df_1

,Unnamed: 0,Barcode,ingredient_list_original,product_name,ingredient_list
0,0,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aqua / water
1,1,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aluminum chlorohydrate
2,2,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,cetearyl alcohol
3,3,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,ceteareth-33
4,4,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,parfum / fragrance
...,...,...,...,...,...
228235,236715,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,10-dimethylundeca-1,5,9-trien-4-ol"
228236,236716,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-amino-m-cresol
228237,236717,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,7-dihydrolinalool"
228238,236718,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-hydroxyindole


In [4]:
#df_1 = df_1[20000:50000]

In [5]:
df_1

,Unnamed: 0,Barcode,ingredient_list_original,product_name,ingredient_list
0,0,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aqua / water
1,1,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,aluminum chlorohydrate
2,2,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,cetearyl alcohol
3,3,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,ceteareth-33
4,4,3600542399326,"Aqua / Water, Aluminum Chlorohydrate, Cetearyl...",Garnier Mineral Hyaluronic Care 72H Deodorant ...,parfum / fragrance
...,...,...,...,...,...
228235,236715,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,10-dimethylundeca-1,5,9-trien-4-ol"
228236,236716,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-amino-m-cresol
228237,236717,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,"6,7-dihydrolinalool"
228238,236718,8436575091013,"2-Ethyl-3,(5 Or 6)-Dimethylpyrazine, 2-Trans, ...",NaN,6-hydroxyindole


In [6]:
ingredient_names = df_1['ingredient_list'].tolist()


In [7]:
df_2 = pd.read_csv('../Data/ingredients.csv')

In [8]:
df_2

,Unnamed: 0,generic_name,synonym
0,0,Niacinamide,niacinamide
1,1,Niacinamide,vitamin b3
2,2,Niacinamide,nicotinamide
3,3,Niacinamide,3-pyridinecarboxamide
4,4,Hyaluronic Acid,hyaluronic acid
...,...,...,...
28819,28819,Parfum,perfum fragrance
28820,28820,Parfum,perfume
28821,28821,Aroma,aroma (flavor)
28822,28822,Aroma,aroma/flavor


In [9]:
synonyms = df_2['synonym'].tolist()


In [10]:
def find_first_match(ingredient, candidates, threshold=0.38):
    ingredient_set = set(ingredient.split())

    # Set to keep track of matched ingredients
    matched_ingredients = set()

    for candidate in candidates:
        # Skip if candidate ingredient has already been matched
        if pd.notna(candidate):  # Check if candidate is not NaN
            if candidate in matched_ingredients:
                continue

            if isinstance(candidate, str):  # Check if candidate is a string
                candidate_set = set(candidate.split())
                union_size = len(ingredient_set.union(candidate_set))

                if union_size != 0:
                    distance = jaccard_distance(ingredient_set, candidate_set)
                    similarity = 1 - distance

                    if similarity > threshold:
                        # Return the first match and exit the function
                        return candidate

                        # Add the matched ingredient to the set
                        matched_ingredients.add(candidate)
    return None

In [11]:
matches_df = pd.DataFrame(columns=['Ingredient', 'Most_Similar_Match'])

# Loop through each ingredient
for ingredient in ingredient_names:
    # Check if the ingredient is a string
    if isinstance(ingredient, str):
        # Find the direct match using list to list check
        direct_match = [synonym for synonym in synonyms if synonym == ingredient] or [None]

        # Use the custom function if the direct match is None
        if direct_match[0] == None:
            match = find_first_match(ingredient, synonyms)
        else:
            match = direct_match[0]

        # Append the result to the DataFrame
        matches_df = matches_df.append({'Ingredient': ingredient, 'Most_Similar_Match': match}, ignore_index=True)

In [12]:
synonym_to_generic = dict(zip(df_2['synonym'], df_2['generic_name']))

# Add a new column to matches_df to store the corresponding generic name
matches_df['Generic_Name'] = None

# Loop through each ingredient
for index, row in matches_df.iterrows():
    ingredient = row['Ingredient']
    match = row['Most_Similar_Match']

    # Check if the match is not None
    if match is not None:
        # Use the dictionary to find the corresponding generic name
        generic_name = synonym_to_generic.get(match)

        # Update the 'Generic_Name' column in matches_df
        matches_df.at[index, 'Generic_Name'] = generic_name

In [13]:
matches_df

,Ingredient,Most_Similar_Match,Generic_Name
0,aqua / water,aqua / water,Water
1,aluminum chlorohydrate,aluminum chlorohydrate,Aluminum Chlorohydrate
2,cetearyl alcohol,cetearyl alcohol,Cetearyl Alcohol
3,ceteareth-33,ceteareth-33,Ceteareth-33
4,parfum / fragrance,parfum / fragrance,Parfum
...,...,...,...
228226,"6,10-dimethylundeca-1,5,9-trien-4-ol","6,10-dimethylundeca-1,5,9-trien-4-ol","6,10-Dimethylundeca-1,5,9-Trien-4-Ol"
228227,6-amino-m-cresol,6-amino-m-cresol,6-Amino-M-Cresol
228228,"6,7-dihydrolinalool","6,7-dihydrolinalool","6,7-Dihydrolinalool"
228229,6-hydroxyindole,6-hydroxyindole,6-Hydroxyindole


In [14]:
none_count = 0
for value in matches_df['Most_Similar_Match']:
    if str(value).strip().lower() == 'none':
        none_count += 1

print(f'The number of occurrences of "None" is: {none_count}')

The number of occurrences of "None" is: 20699


In [15]:
matches_df.to_csv('matches_df_0.38.csv')

In [16]:
matched = round((len(df_1) - none_count)/len(df_1), 2)*100
print(matched, '%')

91.0 %
